In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import time
import scipy
import sklearn
from sklearn import preprocessing
from scipy.io import loadmat
from numpy import linalg as LA
import mkl_algorithms
from mkl_algorithms import OMKR
from mkl_algorithms import OMKRFA
from mkl_algorithms import Raker
from mkl_algorithms import OMKLGF

lam = .001
mat = loadmat('AirData.mat', squeeze_me=True)
X = mat['X']
Y = mat['y']
X = np.transpose(X)
M, N = X.shape

gamma = []
for i in range(0,17):
    gamma.append(10**((i-8)/2))
gamma = np.array(gamma)

time_omkr = np.zeros((1,1))
mse_omkr = np.zeros((M,1))
time_omkrfa = np.zeros((1,200))
mse_omkrfa = np.zeros((M,1))
time_raker = np.zeros((1,200))
mse_raker = np.zeros((M,1))
time_omklgf11 = np.zeros((1,200))
mse_omklgf11 = np.zeros((M,1))
time_omklgf71 = np.zeros((1,200))
mse_omklgf71 = np.zeros((M,1))
time_omklgf101 = np.zeros((1,200))
mse_omklgf101 = np.zeros((M,1))
time_omklgf171 = np.zeros((1,200))
mse_omklgf171 = np.zeros((M,1))
time_omklgf12 = np.zeros((1,200))
mse_omklgf12 = np.zeros((M,1))
time_omklgf14 = np.zeros((1,200))
mse_omklgf14 = np.zeros((M,1))
time_omklgf72 = np.zeros((1,200))
mse_omklgf72 = np.zeros((M,1))
time_omklgf74 = np.zeros((1,200))
mse_omklgf74 = np.zeros((M,1))
n_components = 50


# OMKR
e = np.zeros((M,1))
w = (1/gamma.shape[0])*np.ones((1,gamma.shape[0]))
theta = np.zeros((gamma.shape[0],1))
start_time = time.time()
for i in range(0,M):
    eta = 1/np.sqrt(i+1)
    omkr = OMKR(eta, gamma)
    f_RF, f_RF_p = omkr.predict(X[0:i+1,:], w, theta)
    w, theta = omkr.update(f_RF_p, Y[i], theta, w)
    if i==0:
        e[0] = (f_RF-Y[0])**2
    else:
        e[i] = ( 1/(i+1) )*( (i*e[i-1])+((f_RF-Y[i])**2) )
end_time = time.time()
time_omkr = end_time-start_time
mse_omkr = e

for cc in range(0,200):
    # Generating Random Features
    rbf_feature = np.zeros((N,n_components,gamma.shape[0]))
    for i in range(0,gamma.shape[0]):
        rbf_feature[:,:,i] = np.random.randn(N,n_components)*np.sqrt(gamma[i])
    
    # OMKR-FA
    e = np.zeros((M,1))
    w = (1/gamma.shape[0])*np.ones((1,gamma.shape[0]))
    theta = np.zeros((2*n_components,gamma.shape[0]))
    start_time = time.time()
    for i in range(0,M):
        eta = 1/np.sqrt(i+1)
        omkrfa = OMKRFA(rbf_feature, eta)
        f_RF, f_RF_p, X_features = omkrfa.predict(X[i:i+1,:], theta, w)
        w, theta = omkrfa.update(f_RF_p, Y[i], theta, w, X_features)
        if i==0:
            e[0] = (f_RF-Y[0])**2
        else:
            e[i] = ( 1/(i+1) )*( (i*e[i-1])+((f_RF-Y[i])**2) )
    end_time = time.time()
    time_omkrfa[cc:cc+1] = end_time-start_time
    mse_omkrfa = ( 1/(cc+1) )*( (cc*mse_omkrfa)+e )
    
    # Raker
    e = np.zeros((M,1))
    w = np.ones((1,gamma.shape[0]))
    theta = np.zeros((2*n_components,gamma.shape[0]))
    start_time = time.time()
    for i in range(0,M):
        eta = 1/np.sqrt(i+1)
        raker = Raker(lam, rbf_feature, eta)
        f_RF, f_RF_p, X_features = raker.predict(X[i:i+1,:], theta, w)
        w, theta = raker.update(f_RF_p, Y[i], theta, w, X_features)
        if i==0:
            e[0] = (f_RF-Y[0])**2
        else:
            e[i] = ( 1/(i+1) )*( (i*e[i-1])+((f_RF-Y[i])**2) )
    end_time = time.time()
    time_raker[cc:cc+1] = end_time-start_time
    mse_raker = ( 1/(cc+1) )*( (cc*mse_raker)+e )
    
    # OMKL-GF with M = 1 and J = 1
    e = np.zeros((M,1))
    w = np.ones((1,gamma.shape[0]))
    theta = np.zeros((2*n_components,gamma.shape[0]))
    m = 1 # maximum number of kernels in the chosen subset, M
    J = 1
    ter = 0
    p_c = np.zeros((1,J))
    A_t = np.zeros((gamma.shape[0],J))
    q = np.zeros((gamma.shape[0],1))
    start_time = time.time()
    for i in range(0,M):
        eta = 1/np.sqrt(i+1)
        omklgf = OMKLGF(lam, rbf_feature, gamma, eta, eta, m, J)
        if ter == 0:
            A_t, u, p_c, q = omklgf.graph_gen(w)
        f_RF, f_RF_p, X_features, n_n = omklgf.predict(X[i:i+1,:], theta, w, p_c, A_t)
        w, theta = omklgf.update(f_RF_p, Y[i], theta, w, X_features, n_n, q)
        if i==0:
            e[0] = (f_RF-Y[0])**2
        else:
            e[i] = ( 1/(i+1) )*( (i*e[i-1])+((f_RF-Y[i])**2) )
        if (f_RF-Y[i])**2 < 10**-4:
            ter = 1
    end_time = time.time()
    time_omklgf11[cc:cc+1] = end_time-start_time
    mse_omklgf11 = ( 1/(cc+1) )*( (cc*mse_omklgf11)+e )
    
    # OMKL-GF with M = 7 and J = 1
    e = np.zeros((M,1))
    w = np.ones((1,gamma.shape[0]))
    theta = np.zeros((2*n_components,gamma.shape[0]))
    m = 7
    J = 1
    ter = 0
    p_c = np.zeros((1,J))
    A_t = np.zeros((gamma.shape[0],J))
    q = np.zeros((gamma.shape[0],1))
    start_time = time.time()
    for i in range(0,M):
        eta = 1/np.sqrt(i+1)
        omklgf = OMKLGF(lam, rbf_feature, gamma, eta, eta, m, J)
        if ter == 0:
            A_t, u, p_c, q = omklgf.graph_gen(w)
        f_RF, f_RF_p, X_features, n_n = omklgf.predict(X[i:i+1,:], theta, w, p_c, A_t)
        w, theta = omklgf.update(f_RF_p, Y[i], theta, w, X_features, n_n, q)
        if i==0:
            e[0] = (f_RF-Y[0])**2
        else:
            e[i] = ( 1/(i+1) )*( (i*e[i-1])+((f_RF-Y[i])**2) )
        if (f_RF-Y[i])**2 < 10**-4:
            ter = 1
    end_time = time.time()
    time_omklgf71[cc:cc+1] = end_time-start_time
    mse_omklgf71 = ( 1/(cc+1) )*( (cc*mse_omklgf71)+e )
    
    # OMKL-GF with M = 10 and J = 1
    e = np.zeros((M,1))
    w = np.ones((1,gamma.shape[0]))
    theta = np.zeros((2*n_components,gamma.shape[0]))
    m = 10
    J = 1
    ter = 0
    p_c = np.zeros((1,J))
    A_t = np.zeros((gamma.shape[0],J))
    q = np.zeros((gamma.shape[0],1))
    start_time = time.time()
    for i in range(0,M):
        eta = 1/np.sqrt(i+1)
        omklgf = OMKLGF(lam, rbf_feature, gamma, eta, eta, m, J)
        if ter == 0:
            A_t, u, p_c, q = omklgf.graph_gen(w)
        f_RF, f_RF_p, X_features, n_n = omklgf.predict(X[i:i+1,:], theta, w, p_c, A_t)
        w, theta = omklgf.update(f_RF_p, Y[i], theta, w, X_features, n_n, q)
        if i==0:
            e[0] = (f_RF-Y[0])**2
        else:
            e[i] = ( 1/(i+1) )*( (i*e[i-1])+((f_RF-Y[i])**2) )
        if (f_RF-Y[i])**2 < 10**-4:
            ter = 1
    end_time = time.time()
    time_omklgf101[cc:cc+1] = end_time-start_time
    mse_omklgf101 = ( 1/(cc+1) )*( (cc*mse_omklgf101)+e )
    
    # OMKL-GF with M = 17 and J = 1
    e = np.zeros((M,1))
    w = np.ones((1,gamma.shape[0]))
    theta = np.zeros((2*n_components,gamma.shape[0]))
    m = 17
    J = 1
    ter = 0
    p_c = np.zeros((1,J))
    A_t = np.zeros((gamma.shape[0],J))
    q = np.zeros((gamma.shape[0],1))
    start_time = time.time()
    for i in range(0,M):
        eta = 1/np.sqrt(i+1)
        omklgf = OMKLGF(lam, rbf_feature, gamma, eta, eta, m, J)
        if ter == 0:
            A_t, u, p_c, q = omklgf.graph_gen(w)
        f_RF, f_RF_p, X_features, n_n = omklgf.predict(X[i:i+1,:], theta, w, p_c, A_t)
        w, theta = omklgf.update(f_RF_p, Y[i], theta, w, X_features, n_n, q)
        if i==0:
            e[0] = (f_RF-Y[0])**2
        else:
            e[i] = ( 1/(i+1) )*( (i*e[i-1])+((f_RF-Y[i])**2) )
        if (f_RF-Y[i])**2 < 10**-4:
            ter = 1
    end_time = time.time()
    time_omklgf171[cc:cc+1] = end_time-start_time
    mse_omklgf171 = ( 1/(cc+1) )*( (cc*mse_omklgf171)+e )
    
    # OMKL-GF with M = 1 and J = 2
    e = np.zeros((M,1))
    w = np.ones((1,gamma.shape[0]))
    theta = np.zeros((2*n_components,gamma.shape[0]))
    m = 1
    J = 2
    ter = 0
    p_c = np.zeros((1,J))
    A_t = np.zeros((gamma.shape[0],J))
    q = np.zeros((gamma.shape[0],1))
    start_time = time.time()
    for i in range(0,M):
        eta = 1/np.sqrt(i+1)
        omklgf = OMKLGF(lam, rbf_feature, gamma, eta, eta, m, J)
        if ter == 0:
            A_t, u, p_c, q = omklgf.graph_gen(w)
        f_RF, f_RF_p, X_features, n_n = omklgf.predict(X[i:i+1,:], theta, w, p_c, A_t)
        w, theta = omklgf.update(f_RF_p, Y[i], theta, w, X_features, n_n, q)
        if i==0:
            e[0] = (f_RF-Y[0])**2
        else:
            e[i] = ( 1/(i+1) )*( (i*e[i-1])+((f_RF-Y[i])**2) )
        if (f_RF-Y[i])**2 < 10**-4:
            ter = 1
    end_time = time.time()
    time_omklgf12[cc:cc+1] = end_time-start_time
    mse_omklgf12 = ( 1/(cc+1) )*( (cc*mse_omklgf12)+e )
    
    # OMKL-GF with M = 1 and J = 4
    e = np.zeros((M,1))
    w = np.ones((1,gamma.shape[0]))
    theta = np.zeros((2*n_components,gamma.shape[0]))
    m = 1
    J = 4
    ter = 0
    p_c = np.zeros((1,J))
    A_t = np.zeros((gamma.shape[0],J))
    q = np.zeros((gamma.shape[0],1))
    start_time = time.time()
    for i in range(0,M):
        eta = 1/np.sqrt(i+1)
        omklgf = OMKLGF(lam, rbf_feature, gamma, eta, eta, m, J)
        if ter == 0:
            A_t, u, p_c, q = omklgf.graph_gen(w)
        f_RF, f_RF_p, X_features, n_n = omklgf.predict(X[i:i+1,:], theta, w, p_c, A_t)
        w, theta = omklgf.update(f_RF_p, Y[i], theta, w, X_features, n_n, q)
        if i==0:
            e[0] = (f_RF-Y[0])**2
        else:
            e[i] = ( 1/(i+1) )*( (i*e[i-1])+((f_RF-Y[i])**2) )
        if (f_RF-Y[i])**2 < 10**-4:
            ter = 1
    end_time = time.time()
    time_omklgf14[cc:cc+1] = end_time-start_time
    mse_omklgf14 = ( 1/(cc+1) )*( (cc*mse_omklgf14)+e )
    
    # OMKL-GF with M = 7 and J = 2
    e = np.zeros((M,1))
    w = np.ones((1,gamma.shape[0]))
    theta = np.zeros((2*n_components,gamma.shape[0]))
    m = 7
    J = 2
    ter = 0
    p_c = np.zeros((1,J))
    A_t = np.zeros((gamma.shape[0],J))
    q = np.zeros((gamma.shape[0],1))
    start_time = time.time()
    for i in range(0,M):
        eta = 1/np.sqrt(i+1)
        omklgf = OMKLGF(lam, rbf_feature, gamma, eta, eta, m, J)
        if ter == 0:
            A_t, u, p_c, q = omklgf.graph_gen(w)
        f_RF, f_RF_p, X_features, n_n = omklgf.predict(X[i:i+1,:], theta, w, p_c, A_t)
        w, theta = omklgf.update(f_RF_p, Y[i], theta, w, X_features, n_n, q)
        if i==0:
            e[0] = (f_RF-Y[0])**2
        else:
            e[i] = ( 1/(i+1) )*( (i*e[i-1])+((f_RF-Y[i])**2) )
        if (f_RF-Y[i])**2 < 10**-4:
            ter = 1
    end_time = time.time()
    time_omklgf72[cc:cc+1] = end_time-start_time
    mse_omklgf72 = ( 1/(cc+1) )*( (cc*mse_omklgf72)+e )
    
    # OMKL-GF with M = 7 and J = 4
    e = np.zeros((M,1))
    w = np.ones((1,gamma.shape[0]))
    theta = np.zeros((2*n_components,gamma.shape[0]))
    m = 7
    J = 4
    ter = 0
    p_c = np.zeros((1,J))
    A_t = np.zeros((gamma.shape[0],J))
    q = np.zeros((gamma.shape[0],1))
    start_time = time.time()
    for i in range(0,M):
        eta = 1/np.sqrt(i+1)
        omklgf = OMKLGF(lam, rbf_feature, gamma, eta, eta, m, J)
        if ter == 0:
            A_t, u, p_c, q = omklgf.graph_gen(w)
        f_RF, f_RF_p, X_features, n_n = omklgf.predict(X[i:i+1,:], theta, w, p_c, A_t)
        w, theta = omklgf.update(f_RF_p, Y[i], theta, w, X_features, n_n, q)
        if i==0:
            e[0] = (f_RF-Y[0])**2
        else:
            e[i] = ( 1/(i+1) )*( (i*e[i-1])+((f_RF-Y[i])**2) )
        if (f_RF-Y[i])**2 < 10**-4:
            ter = 1
    end_time = time.time()
    time_omklgf74[cc:cc+1] = end_time-start_time
    mse_omklgf74 = ( 1/(cc+1) )*( (cc*mse_omklgf74)+e )

print('MSE of OMKR is %s' %(mse_omkr[-1]))
print('Execution time of OMKR is %s' %np.mean(time_omkr))
print('MSE of OMKR-FA is %s' %mse_omkrfa[-1])
print('Execution time of OMKR-FA is %s' %np.mean(time_omkrfa))
print('MSE of Raker is %s' %mse_raker[-1])
print('Execution time of Raker is %s' %np.mean(time_raker))
print('MSE of OMKL-GF with M=1 and J=1 is %s' %mse_omklgf11[-1])
print('Execution time of OMKL-GF with M=1 and J=1 is %s' %np.mean(time_omklgf11))
print('MSE of OMKL-GF with M=7 and J=1 is %s' %mse_omklgf71[-1])
print('Execution time of OMKL-GF with M=7 and J=1 is %s' %np.mean(time_omklgf71))
print('MSE of OMKL-GF with M=10 and J=1 is %s' %mse_omklgf101[-1])
print('Execution time of OMKL-GF with M=10 and J=1 is %s' %(np.mean(time_omklgf101)))
print('MSE of OMKL-GF with M=17 and J=1 is %s' %mse_omklgf171[-1])
print('Execution time of OMKL-GF with M=17 and J=1 is %s' %np.mean(time_omklgf171))
print('MSE of OMKL-GF with M=1 and J=2 is %s' %mse_omklgf12[-1])
print('Execution time of OMKL-GF with M=1 and J=2 is %s' %np.mean(time_omklgf12))
print('MSE of OMKL-GF with M=1 and J=4 is %s' %mse_omklgf14[-1])
print('Execution time of OMKL-GF with M=1 and J=4 is %s' %np.mean(time_omklgf14))
print('MSE of OMKL-GF with M=7 and J=2 is %s' %mse_omklgf72[-1])
print('Execution time of OMKL-GF with M=7 and J=2 is %s' %np.mean(time_omklgf72))
print('MSE of OMKL-GF with M=7 and J=4 is %s' %mse_omklgf74[-1])
print('Execution time of OMKL-GF with M=7 and J=4 is %s' %np.mean(time_omklgf74))